In [8]:
import os, shutil, gc

import hda
from getpass import getpass

import xarray as xr
import numpy as np
import cf_xarray
import rioxarray

from datetime import datetime

from cdo import *
cdo = Cdo()
cdo.cleanTempDir()

from matplotlib import pyplot as plt
from matplotlib import colors
from matplotlib import cm
import colormaps as cmo

from cartopy import crs as ccrs
from cartopy import feature as cf
from shapely import geometry
from shapely.geometry import shape

import zipfile

from rich.jupyter import print as rprint
from rich.table import Table
from rich.markdown import Markdown

import warnings
warnings.filterwarnings('ignore')

from tqdm.auto import tqdm


In [9]:
tes = os.path.join(os.getcwd(),'Intro.md')

rprint(Markdown(tes))

/home/jovyan/edskywalker/imber-sen3/Code/Intro.md

In [ ]:
download_dir = os.path.join(os.path.expanduser('~'),"edskywalker", "downloaded-data")
result_dir = os.path.join(os.path.expanduser('~'),"edskywalker","processed-data")

os.makedirs(download_dir, exist_ok=True)
os.makedirs(result_dir, exist_ok=True)

print(download_dir)
rprint(result_dir)

In [ ]:
list_flags_common = ['INLAND_WATER','CLOUD','CLOUD_AMBIGUOUS','CLOUD_MARGIN','INVALID','COSMETIC','SATURATED','SUSPECT','HISOLZEN','HIGHGLINT','SNOW_ICE']
list_flags_process = ['AC_FAIL','WHITECAPS','ADJAC','RWNEG_O2','RWNEG_O3','RWNEG_O4','RWNEG_O5','RWNEG_O6','RWNEG_O7','RWNEG_O8']
list_flags_oc4me = ['OC4ME_FAIL']
list_flags_ocnn = ['OCNN_FAIL']
list_flags_ref01 = []

def flag_data_fast(list_flag, flag_names, flag_values, flag_data, flag_type='WQSF'):
    flag_bits = np.uint64()
    if flag_type == 'SST':
        flag_bits = np.uint8()
    elif flag_type == 'WQSF_lsb':
        flag_bits = np.uint32()
    for flag in list_flag:
        try:
            flag_bits = flag_bits | flag_values[flag_names.index(flag)]
        except:
            print(flag + 'not present')
    return (flag_data & flag_bits) > 0			


In [ ]:
rprint("Please enter your [bold u cyan]username[/bold u cyan] and [bold u cyan]password[/bold u cyan]")

print()

while True:
    try:
        user = input("Enter your WEkEO username: ")
        passw = getpass("Enter your WEkEO password: ")
        c = hda.Client(hda.Configuration(user=user, password=passw), progress=True, max_workers=1)
        print()
        print(f"Login successfull!. Your token is {c.token}.")
        break
    except KeyError:
        print()
        print('You entered wrong username and/or password.')

In [ ]:
# Satellite parameter
rprint("Please enter the Sentinel-3 dataset parameter. For Dataset ID, enter [bold red]1[/bold red] to choose [bold red]EO:EUM:DAT:SENTINEL-3:0556[/bold red] and [bold blue]2[/bold blue] to choose [bold blue]EO:EUM:DAT:SENTINEL-3:OL_2_WFR___[/bold blue].")
print()

while True:
    sat_id = int(input('Satellite ID: '))

    if sat_id == 1:
        dataset_id = 'EO:EUM:DAT:SENTINEL-3:0556'
        print()
        print('Dataset ID: ', dataset_id)
        break
    elif sat_id == 2:
        dataset_id = 'EO:EUM:DAT:SENTINEL-3:OL_2_WFR___'
        print()
        print('Dataset ID: ', dataset_id)
        break
    else:
        print()
        print("You put wrong number. Please try again!")

print()

print("For :satellite: satellite name, enter [bold red]a[/bold red] to choose [bold red]Sentinel-3A[/bold red] and [bold yellow]b[/bold yellow] to choose [bold yellow]Sentinel-3B[/bold yellow]. Leave it blank if you want both Sentinel-3A and Sentinel-3B queried. Please note that Sentinel-3B only available start from 18 May 2018")

print()

sat_nm = input("Satellite name: ")

print()

if sat_nm == 'a' or sat_nm == 'A':
    sat = 'Sentinel-3A'
    print()
    print('Satellite: ', sat)
elif sat_nm == 'b' or sat_nm == 'B':
    sat = 'Sentinel-3B'
    print()
    print('Satellite: ', sat)
else:
    sat = ''
    print()
    print('Both Sentinel-3A and Sentinel-3B will be queried.')


In [ ]:
# Area of interest
rprint('Please input your :earth_asia: area of interest. The coordinates should be in [bold yellow]decimal format[/bold yellow] with :heavy_minus_sign-text: sign for south-of-equator latitude or west-of-greenwich longitude')

north = float(input('North point: ')) # -6.85
south = float(input('South point: ')) # -7.95
west = float(input('West point: ')) # 112.66
east = float(input('East point: ')) # 114.65

bbox = [west, south, east, north]
extent = [west, east, north, south]
bbox_str = f'{west},{east},{south},{north}' 

bbox_polygon = geometry.Polygon(((west,south), (west,north), (east,north), (east,south)))

print()
print(bbox_str)

In [ ]:
# Resolution in meters
resolution = 300
# Approximate conversion from meters to degrees at the equator
# (1 degree of latitude is approximately 111,320 meters)
resolution_degrees = resolution / 111320

# Calculate the number of grid points
num_lon = int(np.ceil((east - west) / resolution_degrees)) + 1
num_lat = int(np.ceil((north - south) / resolution_degrees)) + 1

# Create coordinate arrays
lon = np.linspace(west, east, num_lon)
lat = np.linspace(south, north, num_lat)

In [ ]:
ds = xr.Dataset(
    coords={
        "lon": (["lon"], lon),
        "lat": (["lat"], lat),
    }
)

ds.lat.attrs = {
    'units' : 'degrees_north',
    'unit_long' : "Degrees North",
    'standard_name' : "latitude",
    'long_name' : "Latitude",
    'axis' : 'Y'
}

ds.lon.attrs = {
    'units' : 'degrees_east',
    'unit_long' : "Degrees East",
    'standard_name' : "longitude",
    'long_name' : "Longitude",
    'axis' : 'X'
}

ds["data"] = (["lat", "lon"], np.zeros((num_lat, num_lon)))

ds.rio.write_crs('epsg:4326', inplace=True)

ds.to_netcdf(download_dir + '/grid_data.nc')
dsinput = download_dir + '/grid_data.nc'

grids = cdo.griddes(input=dsinput)

gridfile = download_dir + 'gridfile.txt' 

with open(gridfile, 'w') as f:
    print("\n".join(line.strip("'") for line in grids), file = f)

with open(gridfile, 'r') as file:  # Open in read mode
    content = file.read()

print(content)

In [ ]:
## Time of interest
rprint("Please input your :spiral_calendar: [u]date of interest[/u]. The dates should be in [bold yellow]YYYY-MM-DD[/bold yellow] format.")
print()
dtstart = input('Time start: ')
dtend = input('Time end: ')

In [ ]:
query = {
  "dataset_id": dataset_id, 
  "dtstart": dtstart,
  "dtend": dtend,
  "bbox": bbox,
  "sat": sat,
  "type": "OL_2_WFR___",
  "timeliness": "NT"
}

query_tab = Table(title="Search Query")
query_tab.add_column('Parameter', style='cyan')
query_tab.add_column('Value', style='bright_green')

for col1, col2 in query.items():
    query_tab.add_row(str(col1), str(col2))

rprint(query_tab)

In [ ]:
rprint(Markdown('Please select type of data you want to download. Enter `1` to download geophysical (chlorophyll-a and total suspended matter) or enter `2` to download water surface reflectances.'))

print()

while True:
    parameters = int(input('Parameters: '))

    if parameters == 1:
        print()
        print('Geophysical data will be processed.')
        break
    elif parameters == 2:
        print()
        print('Reflectance data will be processed.')
        break
    else:
        print("You put wrong number. Please try again!")

In [ ]:
search_result = c.search(query)
print(search_result)


In [ ]:
file_id = search_result[0].results[0]['id']
search_result[0].download()

with zipfile.ZipFile(file_id + '.zip', 'r') as zip_ref:
    zip_ref.extractall(download_dir)
#    print(f'Unzipping of product {file_id} finished.')
    os.remove(file_id + '.zip')

filedir = os.path.join(download_dir, file_id)
geo_coords = xr.open_dataset(filedir + '/geo_coordinates.nc')

flag_file = xr.open_dataset(filedir + '/wqsf.nc')
flag_names = flag_file['WQSF'].flag_meanings.split(' ') #flag names
flag_vals = flag_file['WQSF'].flag_masks #flag bit values
flags_data = flag_file.variables['WQSF'].data

start = datetime.strptime(search_result[0].results[0]['properties']['startdate'], '%Y-%m-%dT%H:%M:%S%fZ')
end = datetime.strptime(search_result[0].results[0]['properties']['enddate'], '%Y-%m-%dT%H:%M:%S%fZ')
timestamp = start + (end - start) / 2

dta = xr.Dataset()
dta['longitude'] = geo_coords['longitude']
dta['latitude'] = geo_coords['latitude']

geo_coords.close()
flag_file.close()
gc.collect()


In [ ]:
if parameters == 1:
    keys = ["chl_nn","tsm_nn","chl_oc4me"]
    for k in keys:
        if not k == 'chl_oc4me':
            list_flags = list_flags_common + list_flags_ocnn
        else:
            list_flags = list_flags_common + list_flags_process + list_flags_oc4me

        ds = xr.open_dataset(os.path.join(filedir,f'{k}.nc'))
        dtarr = ds[str(k.upper())].data
        flag_mask = flag_data_fast(list_flags, flag_names, flag_vals, flags_data, flag_type='WQSF')
        
        flagged = np.where(flag_mask, np.nan, dtarr)
        
        dta[str(k)] = xr.DataArray(flagged, dims=('rows','columns'))
        dta[str(k)].attrs = ds[str(k.upper())].attrs

    filename = '_geo'
elif parameters == 2:
    keys = ['Oa01_reflectance','Oa02_reflectance','Oa03_reflectance','Oa04_reflectance','Oa05_reflectance','Oa06_reflectance','Oa07_reflectance','Oa08_reflectance','Oa09_reflectance','Oa10_reflectance','Oa11_reflectance','Oa12_reflectance','Oa16_reflectance','Oa17_reflectance','Oa18_reflectance','Oa21_reflectance']
    list_flags = list_flags_common + list_flags_process
    for k in keys:
        ds = xr.open_dataset(os.path.join(filedir,f'{k}.nc'))
        dtarr = ds[str(k)].data
        flag_mask = flag_data_fast(list_flags, flag_names, flag_vals, flags_data, flag_type='WQSF')
        
        flagged = np.where(flag_mask, np.nan, dtarr)
        
        dta[str(k)] = xr.DataArray(flagged, dims=('rows','columns'))
        dta[str(k)].attrs = ds[str(k)].attrs

    filename = '_ref'

dta = dta.set_coords(['latitude','longitude'])
dta = dta.expand_dims(dim={"time":[timestamp]}, axis=0)

dta = dta.cf.add_bounds(['latitude','longitude'])

In [ ]:
display(dta)

In [ ]:
reggrid = cdo.sellonlatbox(bbox_str, input = dta, returnXDataset = True)

comp = dict(zlib=True, _FillValue=-99999.0, complevel=4)
encoding = {var: comp for var in reggrid.data_vars}

reggrid.to_netcdf(
    os.path.join(download_dir , file_id + str(filename) + '.nc'),
    format='NETCDF4', 
    unlimited_dims=['time'],
    encoding=encoding
)

dataset = xr.open_dataset(os.path.join(download_dir , file_id + str(filename) + '.nc'), decode_coords="all")

display(dataset)

cdo.cleanTempDir()


In [ ]:
fig, ax = plt.subplots(figsize=[10,6], ncols = 3, layout='constrained', subplot_kw=dict(projection=ccrs.Robinson(central_longitude=112.0)))

for i in range(3):
    ax[i].set_extent(extent, crs=ccrs.PlateCarree())
    ax[i].add_feature(cf.LAND.with_scale('10m'), facecolor = 'beige', edgecolor='black', zorder = 1)
#    ax[i].coastlines()
    ini = ax[i].gridlines(draw_labels = True, alpha=0.5)
    ini.top_labels = False
    ini.right_labels = False
    if not i == 0:
        ini.left_labels = False

if parameters == 1:    
    cnn_plot = 10 ** dataset['chl_nn']
    coc_plot = 10 ** dataset['chl_oc4me']
    tsm_plot = 10 ** dataset['tsm_nn']
    
    cnn_plot.plot(ax=ax[0], x='longitude', y='latitude', add_colorbar=False, norm=colors.LogNorm(0.01,100), cmap=cmo.algae, transform=ccrs.PlateCarree(), zorder=0)
    ax[0].set_title("Chlorophyll-a NN")
    coc_plot.plot(ax=ax[1], x='longitude', y='latitude', add_colorbar=False, norm=colors.LogNorm(0.01,100), cmap=cmo.algae, transform=ccrs.PlateCarree(), zorder=0)
    ax[1].set_title("Chlorophyll-a OC4ME")
    tsm_plot.plot(ax=ax[2], x='longitude', y='latitude', add_colorbar=False, norm=colors.LogNorm(0.01,100), cmap=cmo.matter, transform=ccrs.PlateCarree(), zorder=0)
    ax[2].set_title("Total Suspended Matter NN")
    
    cbar1 = plt.colorbar(cm.ScalarMappable(norm=colors.LogNorm(0.01,100), cmap=cmo.algae), shrink=0.7, aspect=40, pad=0.02, orientation = 'horizontal', label = 'Chlorophyll-a Concentration',ax=ax[0:2])
    cbar2 = plt.colorbar(cm.ScalarMappable(norm=colors.LogNorm(0.01,100), cmap=cmo.matter), shrink=0.7, aspect=20, pad=0.02, orientation = 'horizontal', label = 'Total Suspended Matter',ax=ax[2:])

else:
    a06_plot = dataset['Oa06_reflectance']
    a08_plot = dataset['Oa08_reflectance']
    a12_plot = dataset['Oa12_reflectance']
    
    a06_plot.plot(ax=ax[0], x='longitude', y='latitude', add_colorbar=True, norm=colors.Normalize(0,0.1), cmap=cmo.hawaii, transform=ccrs.PlateCarree(), zorder=0)
    ax[0].set_title("Oa06")
    a08_plot.plot(ax=ax[1], x='longitude', y='latitude', add_colorbar=True, norm=colors.Normalize(0,0.025), cmap=cmo.hawaii, transform=ccrs.PlateCarree(), zorder=0)
    ax[1].set_title("Oa08")
    a12_plot.plot(ax=ax[2], x='longitude', y='latitude', add_colorbar=True, norm=colors.Normalize(0,0.05), cmap=cmo.hawaii, transform=ccrs.PlateCarree(), zorder=0)
    ax[2].set_title("Oa12")

    

In [ ]:
reggridded = cdo.remapcon(gridfile, input = dataset, returnXDataset = True)

comp = dict(zlib=True, _FillValue=-99999.0, complevel=4)
encoding = {var: comp for var in reggridded.data_vars}

reggridded.to_netcdf(
    os.path.join(result_dir , file_id + str(filename) + '.nc'),
    format='NETCDF4', 
    unlimited_dims=['time'],
    encoding=encoding
)

dataset = xr.open_dataset(os.path.join(result_dir , file_id + str(filename) + '.nc'), decode_coords="all")

display(dataset)

cdo.cleanTempDir()

In [ ]:
fig, ax = plt.subplots(figsize=[10,6], ncols = 3, layout='constrained', subplot_kw=dict(projection=ccrs.Robinson(central_longitude=112.0)))

for i in range(3):
    ax[i].set_extent(extent, crs=ccrs.PlateCarree())
    ax[i].add_feature(cf.LAND.with_scale('10m'), facecolor = 'beige', edgecolor='black', zorder = 1)
#    ax[i].coastlines()
    ini = ax[i].gridlines(draw_labels = True, alpha=0.5)
    ini.top_labels = False
    ini.right_labels = False
    if not i == 0:
        ini.left_labels = False

if parameters == 1:    
    cnn_plot = 10 ** dataset['chl_nn']
    coc_plot = 10 ** dataset['chl_oc4me']
    tsm_plot = 10 ** dataset['tsm_nn']
    
    cnn_plot.plot(ax=ax[0], add_colorbar=False, norm=colors.LogNorm(0.01,100), cmap=cmo.algae, transform=ccrs.PlateCarree(), zorder=0)
    ax[0].set_title("Chlorophyll-a NN")
    coc_plot.plot(ax=ax[1], add_colorbar=False, norm=colors.LogNorm(0.01,100), cmap=cmo.algae, transform=ccrs.PlateCarree(), zorder=0)
    ax[1].set_title("Chlorophyll-a OC4ME")
    tsm_plot.plot(ax=ax[2], add_colorbar=False, norm=colors.LogNorm(0.01,100), cmap=cmo.matter, transform=ccrs.PlateCarree(), zorder=0)
    ax[2].set_title("Total Suspended Matter NN")
    
    cbar1 = plt.colorbar(cm.ScalarMappable(norm=colors.LogNorm(0.01,100), cmap=cmo.algae), shrink=0.7, aspect=40, pad=0.02, orientation = 'horizontal', label = 'Chlorophyll-a Concentration',ax=ax[0:2])
    cbar2 = plt.colorbar(cm.ScalarMappable(norm=colors.LogNorm(0.01,100), cmap=cmo.matter), shrink=0.7, aspect=20, pad=0.02, orientation = 'horizontal', label = 'Total Suspended Matter',ax=ax[2:])

else:
    a06_plot = dataset['Oa06_reflectance']
    a08_plot = dataset['Oa08_reflectance']
    a12_plot = dataset['Oa12_reflectance']
    
    a06_plot.plot(ax=ax[0], add_colorbar=True, norm=colors.Normalize(0,0.1), cmap=cmo.hawaii, transform=ccrs.PlateCarree(), zorder=0)
    ax[0].set_title("Oa06")
    a08_plot.plot(ax=ax[1], add_colorbar=True, norm=colors.Normalize(0,0.025), cmap=cmo.hawaii, transform=ccrs.PlateCarree(), zorder=0)
    ax[1].set_title("Oa08")
    a12_plot.plot(ax=ax[2], add_colorbar=True, norm=colors.Normalize(0,0.05), cmap=cmo.hawaii, transform=ccrs.PlateCarree(), zorder=0)
    ax[2].set_title("Oa12")

    

In [ ]:
gc.collect()

for allitem in os.listdir(download_dir):
    path = os.path.join(download_dir,allitem)
    if os.path.isfile(path):
        os.remove(path)
    elif os.path.isdir(path):
        shutil.rmtree(path)
